In [11]:
import os

import shutil

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F

import torchvision
from torchvision import models, datasets, transforms
from torchvision.datasets import ImageFolder

from PIL import Image
import matplotlib.pyplot as plt

##### Sort Dataset Into Folders ####

In [14]:
base_dir = "Cat Vs Dog Dataset"
train_dir = os.path.join(base_dir,"train")
test_dir = os.path.join(base_dir,"test1")

if not os.path.exists(f"{train_dir}/cat"):
        os.makedirs(f"{train_dir}/cat")
        
if not os.path.exists(f"{train_dir}/dog"):
        os.makedirs(f"{train_dir}/dog")
        
if not os.path.exists(f"{test_dir}/cat"):
        os.makedirs(f"{test_dir}/cat")
        
if not os.path.exists(f"{test_dir}/dog"):
        os.makedirs(f"{test_dir}/dog")

onlyFiles = [filename for filename in os.listdir(train_dir) if not os.path.isdir(os.path.join(train_dir,filename))

for filename in onlyFiles:
        
        label = filename.split()[0]
        
        if(label == "cat"):
                shutil.move(f"{train_dir}/{filename}" , f"{train_dir}/cat/{filename}")
                
        elif(label == "dog"):
                shutil.move(f"{train_dir}/{filename}" , f"{train_dir}/dog/{filename}")
        
        else:
                print("NO Label")

print(os.listdir(train_dir))


TypeError: 'module' object is not callable